In [ ]:
# a simple algorithm to update stop price


# region imports
from AlgorithmImports import *
# endregion

class Premonition(QCAlgorithm):

    def Initialize(self):
        
        self.SetCash(100000)
        
        self.SetStartDate(2007,1,1)
        self.SetEndDate(2010,1,1)
        
        
        self.citi = self.AddEquity("C", Resolution.Daily) 
        self.citi.SetDataNormalizationMode(DataNormalizationMode.Raw)
        self.c = self.citi.Symbol
        
        self.sellTicket = None
        self.invest = True
        self.loss_limit = 0.5
        self.first_day_close = None

    def OnData(self, data):
        
        if not self.Portfolio.Invested and self.invest:
            # Initial Market Order for C
            self.initial_order = self.MarketOrder(self.c, 100)
            self.first_day_close = self.Securities["C"].Close
            self.invest = False
        
        
        if self.sellTicket is None:
            # Initiate a Market Order if Stop of 50% loss is hit.
            self.sellTicket = self.StopMarketOrder(
                self.c,
                -100,
                self.Securities["C"].Close * self.loss_limit,
                "Loss Limit Stop")


    def OnEndOfDay(self):
        '''End of each day, report the current Close Price'''
        if self.sellTicket is None:
            return

        else:
            self.Log(str(self.Securities["C"].Close))

        # 5% drop from Open --> Close
        # Update the Stop Price --> 50% stop loss --> 75% stop loss
        if ((self.Securities["C"].Open - self.Securities["C"].Close) / self.Securities["C"].Open) > 0.05:
            updateSettings = UpdateOrderFields()
            updateSettings.StopPrice = self.first_day_close * 0.75
            response = self.sellTicket.Update(updateSettings)

            if response.IsSuccess:
                self.Debug("Order was Updated!")


    def OnOrderEvent(self, OrderEvent):
        # Event when the order is filled. Debug log the order fill. :OrderEvent:

        if OrderEvent.FillQuantity == 0:
            return
        
        fetched = self.Transactions.GetOrderById(OrderEvent.OrderId)

        self.Log(f"{str(fetched.Type)} was filled. Symbol: {str(OrderEvent.Symbol)}. Quantity: {str(OrderEvent.FillQuantity)}. Direction: {str(OrderEvent.Direction)}")